In [1]:
import numpy as np
import torch
import torch.nn as nn
import math
from torch.autograd import Variable

# 1. Input Embedding 输入嵌入

In [2]:
# 该类继承nn.Module, 这样就有标准层的一些功能, 这里我们也可以理解为一种模式, 我们自己实现的所有层都会这样去写.
class Embeddings(nn.Module):
    def __init__(self, vocab, d_model):
        """类的初始化函数, 有两个参数, d_model: 指词嵌入的维度, 默认d_model=512, vocab: 指词表的大小."""
        # 使用super的方式指明继承nn.Module的初始化函数, 使得新建class各种新模块初始化之前，先把super类模块初始化。
        # 之后我们自己实现的所有层都会这样去写，算是一个框架
        super(Embeddings, self).__init__()
        
        # 之后就是调用nn中的预定义层Embedding, 获得一个词嵌入对象self.lut.在NLP中，lut是lookup table的缩写
        # 这个lut矩阵的大小是 vocab_size * d_model
        self.lut = nn.Embedding(vocab, d_model)
        # 最后就是将d_model传入类中
        self.d_model = d_model

    def forward(self, x):
        """可以将其理解为该层的前向传播逻辑，所有层中都会有此函数，这也算是一种框架型写法
           当把参数传给该类的实例化对象时, 自动调用该forward函数
           参数x: 因为Embedding层是首层, 所以代表输入给模型的文本通过词汇映射后的张量"""

        # 将x传给self.lut并与根号下self.d_model相乘作为结果返回
        # 即 y1=self.lut(x)
        """ This is a common practice to scale the embeddings
            特别是在tansformer模型中，input embedding后的向量 y1 还要加上位置编码，才能输入到encoder层中
            而当位置编码较大的时候，容易把y1的信息淹没。为了避免丢失有关信息，我们需要在一定程度上放大y1
            常见的方法就是乘上 根号(d_model)，
        """
        return self.lut(x) * math.sqrt(self.d_model)


nn.Embedding的演示

In [3]:
# input tensor中的每一个数字，都会被映射成一个长度=d_model的向量
# vocab只要大于input长度就行，比如这里vocab>3
emb = Embeddings(vocab=100,d_model=5)

# Embeddings()只接受tensor作为input，而不能直接是nparray
x = torch.LongTensor([[1.0,0.0,1.0],[0.0,2.0,0.0],[3.0,1.5,2.0]])

# 在Pytorch的class中，当我们执行 emb(x)时，它会自动调用执行forward()函数，这也是pytorch框架的规则之一，等同于 emb.forward(x)
"""When you create an instance of a PyTorch module (such as your Embeddings class), the forward() method is implicitly
invoked when you call the instance with input data. Instead of explicitly writing emb.forward(x), you can directly use emb(x).
This automatic invocation simplifies code."""
emb(x)

tensor([[[ 1.9196,  0.4591,  0.4439,  0.3286, -0.9220],
         [-1.9628, -2.4688,  1.6908, -1.8594, -4.2726],
         [ 1.9196,  0.4591,  0.4439,  0.3286, -0.9220]],

        [[-1.9628, -2.4688,  1.6908, -1.8594, -4.2726],
         [-2.7864,  1.7916,  2.5924,  1.6822,  0.6212],
         [-1.9628, -2.4688,  1.6908, -1.8594, -4.2726]],

        [[-0.1518, -0.4172, -1.9871,  0.1049,  1.7251],
         [ 1.9196,  0.4591,  0.4439,  0.3286, -0.9220],
         [-2.7864,  1.7916,  2.5924,  1.6822,  0.6212]]],
       grad_fn=<MulBackward0>)

In [4]:
embedding = nn.Embedding(100, 6)
kk = torch.LongTensor([[3,1,6],[0,2,5]])
embedding(kk)

tensor([[[-1.2827e-01, -1.3691e+00,  1.4840e-01,  2.2584e+00,  1.7436e-03,
           5.6337e-01],
         [-3.1070e-01, -8.9626e-02, -1.4286e+00, -6.6563e-01,  6.5881e-01,
          -9.9134e-01],
         [ 1.5717e+00,  1.3316e+00,  1.3836e+00,  1.1752e+00, -3.3995e-01,
           4.4781e-01]],

        [[ 1.2836e+00,  5.8506e-01, -2.2457e+00,  1.4369e+00,  1.1909e+00,
          -3.7735e-01],
         [-2.1591e-01, -1.3198e+00,  1.4246e+00,  7.8521e-01, -9.3659e-01,
          -9.9296e-01],
         [ 8.3780e-01, -5.0898e-01,  3.5092e-01,  3.6714e-01, -2.5461e-01,
           2.0625e-01]]], grad_fn=<EmbeddingBackward0>)

nn.Embeddings 里num_embeddings(即vocab)参数的解释：
其中，num_embeddings代表词典大小尺寸，比如训练时所可能出现的词语一共5000个词，那么就有num_embedding=5000。
NLP中是这样的，先准备一个dictionary，里面包含了所有可能出现的字，然后给一句话，对应生成one-hot encoding。
这里只要保证 num_embeddings ≥ 可能出现的字的数量就行。

# 2. Positional Encoding位置编码

In [5]:
# 定义位置编码器类, 我们同样把它看做一个层, 因此会继承nn.Module    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        """注意，【d_model需要为偶数】，不然会导致报错。max_len: 每个句子的最大长度"""
        super(PositionalEncoding, self).__init__()

        # 实例化nn中预定义的Dropout层, 并将dropout传入其中, 获得对象self.dropout
        # the dropout layer randomly sets a fraction of the elements in x to 0 (according to the specified dropout probability)
        # it is a way of regularization to avoid overfitting, ususally set dropout= 0.1-0.5
        self.dropout = nn.Dropout(p=dropout)

        # 初始化一个位置编码矩阵, 它是一个0阵，矩阵的大小是max_len * d_model.
        pe = torch.zeros(max_len, d_model)

        # 初始化一个绝对位置矩阵, 我们首先使用arange方法获得一个连续自然数向量，然后再使用unsqueeze方法拓展向量维度使其成为矩阵， 
        # 又因为参数传的是1，代表矩阵拓展的位置，会使向量变成一个 (max_len * 1) 的矩阵
        position = torch.arange(0, max_len).unsqueeze(1)

        
        ## 位置编码的实现其实很简单，直接对照着公式去敲代码就可以，下面这个代码只是其中一种实现方式；
        ## 从理解来讲，需要注意的就是偶数和奇数在公式上有一个共同部分，即一个1*d_model形状的变换矩阵div_term
        ## 我们使用log函数把次方拿下来，方便计算
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # pe现在还只是一个二维矩阵，要拓展一个维度，才能和embedding的输出（一个三维张量）相加， pe.size() = (1,maxlen,d_model)
        pe = pe.unsqueeze(0)

        # 最后把pe位置编码矩阵注册成模型的buffer，什么是buffer呢，
        # 我们把它认为是对模型效果有帮助的，但是却不是模型结构中超参数或者参数，不需要随着优化步骤进行更新的增益对象. 
        # 注册之后我们就可以在模型保存后重加载时和模型结构与参数一同被加载。这也是一种框架性的东西
        self.register_buffer('pe', pe)

    def forward(self, x):
        """forward函数的参数是x, 表示文本序列的词嵌入表示"""
        # 在相加之前我们对pe做一些适配工作， 将这个三维张量的第二维也就是max_len的那一维变成和x的第二维相同，即x.size(1)，
        # 这样一来，pe和x的size(1), size(2) 都相同，才能相加
        # 最后使用Variable进行封装，使其与x的样式相同，但是它是不需要进行梯度求解的，因此把requires_grad设置成false.
        x = x + Variable(self.pe[:, :x.size(1)],requires_grad=False)
        # 最后使用self.dropout对象进行'丢弃'操作, 并返回结果.
        return self.dropout(x)

In [6]:
"""【注意，d_model需要为偶数，不然会使得奇数位和偶数位向量不一样长，导致报错】"""
vocab = 100
d_model = 6
dropout = 0.1
max_len = 200

emb = Embeddings(vocab, d_model)
pe = PositionalEncoding(d_model, dropout, max_len)
x = torch.LongTensor([[1,0,1,0],[0,0,2,3],[3,1,5,2]])

pe(emb(x))

tensor([[[-1.3375, -0.0682, -0.6525,  0.0000, -2.7479,  4.9678],
         [ 2.0500,  3.1669, -0.0000,  3.8381, -1.6612, -2.9135],
         [-0.3272, -1.6417, -0.5495,  0.4416, -2.7431,  4.9678],
         [ 1.2719,  1.4665, -1.5695,  3.8286, -1.6564, -2.9136]],

        [[ 1.1151,  3.6776, -1.7237,  3.8393, -1.6636, -2.9135],
         [ 2.0500,  3.1669, -1.6721,  3.8381, -1.6612, -2.9135],
         [ 4.1564,  2.3259,  0.7435, -0.7137,  2.9358,  2.7561],
         [ 2.3496,  2.3266, -4.4789,  4.3681,  6.0578, -0.2104]],

        [[ 2.1928,  0.0000, -4.6332,  4.3788,  6.0506, -0.2104],
         [-0.4025, -0.5790, -0.0000,  0.4452, -2.7455,  4.9678],
         [ 3.6119,  2.0083, -3.0000,  1.9317, -0.4141,  1.7205],
         [ 3.3029,  1.6883,  0.7947, -0.7197,  2.9382,  2.7561]]],
       grad_fn=<MulBackward0>)

维度演示

In [7]:
x = embedding(kk)
print(x.shape)

pe = torch.zeros(100, 6) # suppose max_len=100, d_model=6
position = torch.arange(0, 100).unsqueeze(1)
div_term = torch.exp(torch.arange(0, 6, 2) * -(math.log(10000.0) / 6))
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)

pe = pe.unsqueeze(0)
print(pe.size())
print(pe[:, :x.size(1)].size()) # 所以最后 x+ pe[:, :x.size(1)] 是 ([2, 3, 6]) + ([1, 3, 6])

torch.Size([2, 3, 6])
torch.Size([1, 100, 6])
torch.Size([1, 3, 6])


In [8]:
# x = torch.randint(0, 8, (2,3,6))
x = torch.tensor([[[4, 5, 2, 3, 5, 1],
                   [0, 1, 2, 2, 4, 3],
                   [2, 5, 0, 6, 0, 5]],
                  
                    [[2, 4, 6, 7, 7, 2],
                     [2, 2, 3, 6, 7, 6],
                     [0, 1, 4, 1, 3, 3]]])

# pe = torch.randint(0, 5, (1,3,6))
pe = torch.tensor([[[1, 4, 2, 0, 0, 0],
         [1, 3, 4, 0, 2, 4],
         [3, 1, 0, 3, 3, 1]]])

print(x.size(),pe.size())
print(x + pe)

del x, pe

torch.Size([2, 3, 6]) torch.Size([1, 3, 6])
tensor([[[ 5,  9,  4,  3,  5,  1],
         [ 1,  4,  6,  2,  6,  7],
         [ 5,  6,  0,  9,  3,  6]],

        [[ 3,  8,  8,  7,  7,  2],
         [ 3,  5,  7,  6,  9, 10],
         [ 3,  2,  4,  4,  6,  4]]])


# 3. Subsequent_mask 掩码张量
在 transformer 中，掩码张量的主要作用在应用 attention (将在下一小节讲解) 时，有一些生成的 attention 张量中的值计算有可能已知了未来信息而得到的，未来信息被看到是因为训练时会把整个输出结果都一次性进行 Embedding，但是理论上解码器的的输出却不是一次就能产生最终结果的，而是一次次通过上一次结果综合得出的，因此，未来的信息可能被提前利用。所以，我们会进行遮掩。

这里遮掩的意思是，未来数据全部*0， 过往数据*1.所以要形成一个斜对角右边为0，左边为1的矩阵

In [9]:
def subsequent_mask(size):
    attn_shape = (1, size, size)

    # 然后使用np.ones方法向这个形状中添加1元素,形成上三角阵, 最后为了节约空间, 
    # 再使其中的数据类型变为无符号8位整形unit8 
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')

    # 最后将三角阵反转
    return torch.from_numpy(1 - subsequent_mask)

In [10]:
size = 5
sm = subsequent_mask(size)
print(sm.size())
sm

torch.Size([1, 5, 5])


tensor([[[1, 0, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [1, 1, 1, 0, 0],
         [1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1]]], dtype=torch.uint8)

In [11]:
# 三角阵函数 triu详解
jj = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])

print(np.triu(jj, k=-1))
print(np.triu(jj, k=0))
print(np.triu(jj, k=1))


[[ 1  2  3]
 [ 4  5  6]
 [ 0  8  9]
 [ 0  0 12]]
[[1 2 3]
 [0 5 6]
 [0 0 9]
 [0 0 0]]
[[0 2 3]
 [0 0 6]
 [0 0 0]
 [0 0 0]]


# 4.1 Attention 注意力机制
什么是注意力:
我们观察事物时，之所以能够快速判断一种事物 (当然允许判断是错误的), 是因为我们大脑能够很快把注意力放在事物最具有辨识度的部分从而作出判断，而并非是从头到尾的观察一遍事物后，才能有判断结果。正是基于这样的理论，就产生了注意力机制。

注意力机制模仿了生物观察行为的内部过程，即一种将内部经验和外部感觉对齐从而增加部分区域的观察精细度的机制。【注意力机制可以快速提取稀疏数据的重要特征】，因而被广泛用于自然语言处理任务，特别是机器翻译。而自注意力机制是注意力机制的改进，其减少了对外部信息的依赖，更擅长捕捉数据或【特征的内部相关性】。

注意力计算规则:
它需要三个指定的输入 Q (query), K (key), V (value), 然后通过公式得到注意力的计算结果，这个结果代表 query 在 key 和 value 作用下的表示。

Q 是一段准备被概括的文本；K 是给出的提示；V 是大脑中的对提示 K 的延伸。

当 Q=K=V 时，称作自注意力机制.

In self-attention, each token in the input sequence is associated with a set of attention weights. These determine how much importance the token should place on each other token in the sequence.

Attention机制其实就是一系列注意力分配系数，也就是一系列权重参数的计算罢了。

In [12]:
# 注意力机制的代码规则
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, query, key, value, mask=None, dropout=None):
        """注意力机制的实现, 
        Q.size() = [batch_size * n_heads * len_q * d_k]  
        K.size() =  [batch_size * n_heads * len_k * d_k]   
        V.size() = [batch_size * n_heads * len_k * d_v]

        Q.dot(Kt) = [batch_size x n_heads x len_q x len_k] 
        """
        # 按照注意力公式, 将query与key的转置相乘, 这里面key是将最后两个维度进行转置, 再除以缩放系数根号下d_k,
        
        d_k = query.size(-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

        # 接着判断是否使用掩码张量。如是，使用tensor的masked_fill方法, 
        # 将掩码张量中为0的部分换成-1e9（一个无限小的值），softmax之后基本就是0，对q的单词不起作用
        """
        注意，atten_mask如果不为 None的话， 它的形状必须与 Scores一样。
        所以在此处，attn_mask.size = [batch_size * len_q * len_k]
        """  
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # 对scores的最后一维进行softmax操作, 这样获得最终的注意力张量
        p_attn = nn.Softmax(dim=-1)(scores)
        """也可以用以下写法，结果完全一致
        import torch.nn.functional as F
        F.softmax(scores, dim = -1) """
        
        # 之后判断是否使用dropout进行随机置0
        if dropout is not None:
            p_attn = dropout(p_attn)

        # 最后, 根据公式将p_attn与value张量相乘获得最终的query注意力表示, 同时返回注意力张量
        content = torch.matmul(p_attn, value)
        return content, p_attn

实例化尝试

In [13]:
vocab = 100
d_model = 6
dropout = 0.1
max_len = 200

emb = Embeddings(vocab, d_model)
pe = PositionalEncoding(d_model, dropout, max_len)
x = torch.LongTensor([[1,0,1,0],[0,0,2,3],[3,1,5,2]])

pe_result = pe(emb(x))
Q = K = V = pe_result

ScaledDotProductAttention()(Q, K, V)

(tensor([[[ 4.6026e+00,  1.0878e+00, -1.9334e+00,  2.4451e+00, -1.1223e+00,
            1.1980e+00],
          [-1.0332e+00,  7.6754e-01,  1.3017e+00, -4.4676e-01,  8.0331e-01,
            2.2941e-02],
          [ 4.8816e+00,  6.5325e-01, -1.9050e+00,  2.4438e+00, -1.1210e+00,
            1.1980e+00],
          [-1.6076e+00, -2.7890e-01,  1.4275e+00, -5.0885e-01,  2.2842e+00,
            7.4942e-05]],
 
         [[-1.6190e+00,  1.2292e-01,  1.3158e-01, -4.9648e-01,  2.3674e+00,
           -1.3288e+00],
          [-3.3594e-01,  1.0877e+00,  1.0801e+00, -4.9727e-01,  2.3732e+00,
           -2.7105e-01],
          [-3.4067e+00, -5.3229e+00, -5.7632e-01,  2.6492e-01, -5.0803e+00,
            1.0105e-01],
          [-4.7074e-01, -4.5503e+00,  6.9673e-01,  7.2056e-01, -4.1788e-01,
            2.7841e+00]],
 
         [[-3.8928e+00, -5.6165e+00, -4.1549e-01,  3.0756e-01, -4.7662e+00,
            3.7173e-01],
          [ 5.5521e+00,  3.5966e-01, -1.3843e+00,  2.3226e+00, -9.8365e-01,
         

In [14]:
# 加上掩码张量
sm = subsequent_mask(Q.size(1))
print(sm.size())  

# 当前sm.size = [1 * len_q * len*k]， 要让attention_mask的形状等于 [batch_size * len_q * len*k]
sm = sm.repeat(Q.size(0), 1, 1)
ScaledDotProductAttention()(Q, K, V, sm )

torch.Size([1, 4, 4])


(tensor([[[ 4.0243e+00,  1.9885e+00, -1.9924e+00,  2.4478e+00, -1.1250e+00,
            1.1980e+00],
          [-7.5211e-01,  1.3654e+00,  1.2546e+00, -4.3359e-01, -2.5300e-02,
            2.6941e-02],
          [ 4.8816e+00,  6.5325e-01, -1.9050e+00,  2.4438e+00, -1.1210e+00,
            1.1980e+00],
          [-1.6076e+00, -2.7890e-01,  1.4275e+00, -5.0885e-01,  2.2842e+00,
            7.4942e-05]],
 
         [[-1.7970e+00,  0.0000e+00,  0.0000e+00, -4.9868e-01,  2.3758e+00,
           -1.4822e+00],
          [-3.3617e-01,  1.0983e+00,  1.0806e+00, -4.9965e-01,  2.3777e+00,
           -2.7728e-01],
          [-3.4067e+00, -5.3229e+00, -5.7632e-01,  2.6492e-01, -5.0803e+00,
            1.0105e-01],
          [-4.7074e-01, -4.5503e+00,  6.9673e-01,  7.2056e-01, -4.1788e-01,
            2.7841e+00]],
 
         [[-3.6439e-01, -2.2699e+00,  6.5661e-01,  7.7216e-01, -1.7396e+00,
            3.0246e+00],
          [ 4.9592e+00,  1.4777e+00, -1.9408e+00,  2.4466e+00, -1.1226e+00,
         

# 4.2 Multihead Attention 多头注意力机制

对 Q，K，V 分别进行线性变换。得到输出结果后，多头的作用才开始显现，每个头开始从词义层面分割输出的张量，也就是只分割了最后一维的词嵌入向量 d_model。然后将每个头的线性变换结果，输入到注意力机制（scaled dot product attention）中，就形成多头注意力机制.

In [15]:
# import image module 
from IPython.display import Image 
Image(url="多头注意力机制结构图.png", width=250, height=250)

In [16]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads,  dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        
        self.n_heads = n_heads
        # 在函数中，首先使用了一个测试中常用的assert语句，判断h是否能被d_model整除，
        assert d_model % self.n_heads == 0
        
        # 得到每个头获得的分割词向量维度d_k
        self.d_k = self.d_v = d_model // self.n_heads

        ## 输入进来的QKV是相等的，特别是默认 d_q = d_k
        # 我们会使用映射linear做一个映射得到参数矩阵Wq, Wk,Wv ，这三个线性矩阵的维度都是 d_model*d_model，作用于QKV矩阵后，不改变其形状
        # 然后还有最后一个linear层，作用于把 Q,K,V 计算出的attention值
        
        self.W_Q = nn.Linear(d_model, self.d_k * self.n_heads) # d_q = d_k
        self.W_K = nn.Linear(d_model, self.d_k * self.n_heads)
        self.W_V = nn.Linear(d_model, self.d_v * self.n_heads)
        self.linear = nn.Linear(self.n_heads * self.d_v, d_model)
        
        # 层后残差连接与规范化:  LayerNormalization (x + Sublayer(x))
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, attn_mask=None):
        ##输入进来的数据形状： Q: [batch_size * len_q * d_model], K: [batch_size * len_k * d_model], V: [batch_size * len_v * d_model]
        residual = query
        batch_size =  query.size(0)
        
        ## 然后把Q, K, V都分成n个heads，即从shape [batch_size * len_q * d_model] 变成 [batch_size * len_q * nheads * d_q]
        ## 前两维不变（第二维的-1保留原有维度），拆分第三维d_model。最后再将第2、3维调换。 如 q_s.size =  [batch_size * nheads * len_q * d_q]
        ## 第2、3维转置是因为，这样能让代表句子长度的len_q和词向量维度d_k相邻，这样注意力机制才能找到词义与句子位置的关系

        q_s = self.W_Q(query).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
    
        k_s = self.W_K(key).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        
        v_s = self.W_V(value).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]
        
        """
        注意，atten_mask如果不为 None的话， 它的形状必须与 Scores一样。
        所以在此处，attn_mask.size = [batch_size * n_heads * len_q * len_k]
        """          
        # 此处计算后，content.size = q_s.size;   attn.size = [batch_size x n_heads x len_q x len_k]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, mask=attn_mask, dropout=self.dropout)

        # 通过多头注意力计算后，我们就得到了每个头计算结果组成的4维张量，我们需要将其转换为输入的形状以方便后续的计算，
        # 因此这里开始进行第一步处理环节的逆操作，先对第二和第三维进行转置，然后使用contiguous方法，
        # 这个方法的作用就是能够让转置后的张量应用view方法，否则将无法直接使用。接着就是使用view重塑形状，变成和输入形状相同.
        # 此时 context.size = Q.size = [batch_size * len_q * d_model]
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v) 
        output = self.linear(context)

        # 最后是残差连接与层规范化:  LayerNormalization (x + Sublayer(x))，目的是避免梯度消失或爆炸
        output = self.layer_norm(output + residual)
        return output, attn

多头注意力层运行示例：

In [36]:
mha = MultiHeadAttention(d_model=6, n_heads=3)

mha(Q, K, V)[0]

tensor([[[ 1.0462,  0.9504, -1.3156,  0.7360, -1.3109, -0.1061],
         [-1.2725,  1.6924,  0.9379, -0.3580, -0.5667, -0.4331],
         [ 1.3677,  0.4114, -1.3766,  0.8972, -1.1057, -0.1939],
         [-1.3827, -0.6134,  1.1056, -0.5626,  1.4740, -0.0210]],

        [[-0.9869,  0.1937,  0.2879,  0.1931,  1.7085, -1.3962],
         [-0.5755,  0.2375,  1.2140, -1.2852,  1.3007, -0.8915],
         [-0.5830, -1.1166,  0.7646,  1.2922, -1.2021,  0.8449],
         [-0.2621, -1.9554,  0.4849,  0.4770, -0.0266,  1.2823]],

        [[-0.5506, -0.7085, -0.1370,  0.9468, -1.2209,  1.6701],
         [ 1.2339,  0.6990, -1.3821,  0.8510, -1.1321, -0.2696],
         [ 1.9352, -0.6512, -0.3518,  0.6522, -0.9070, -0.6774],
         [-0.7978, -0.9497,  0.6214,  1.4509, -1.1417,  0.8168]]],
       grad_fn=<NativeLayerNormBackward0>)

In [18]:
# 加上掩码张量
sm = subsequent_mask(4)
print(sm.size())
sm = sm.unsqueeze(0).repeat(3,3,1,1)  # 必须让 sm.size 和Scores一样
print(sm.size())

mha(Q, K, V, sm)

torch.Size([1, 4, 4])
torch.Size([3, 3, 4, 4])


(tensor([[[ 1.4564,  0.6518, -1.0835,  0.7853, -1.0176, -0.7924],
          [-1.5903,  0.5243,  1.6777, -0.5850,  0.1261, -0.1529],
          [ 1.8312, -0.2869, -1.1017,  0.7246, -0.8524, -0.3148],
          [-1.4676, -0.6761,  1.0439, -0.5549,  1.3852,  0.2696]],
 
         [[-0.9725, -0.1947,  0.6826, -0.7994,  1.8844, -0.6004],
          [-0.8585,  0.1855,  1.1382, -1.5356,  1.2430, -0.1727],
          [-0.4704, -1.3399,  0.8800,  0.8734, -1.0771,  1.1340],
          [-0.1126, -1.9128,  0.4389,  0.1833, -0.0460,  1.4492]],
 
         [[-0.0668, -1.1417, -0.0916,  0.2697, -0.9131,  1.9435],
          [ 1.7038,  0.4109, -1.3223,  0.4560, -0.9344, -0.3140],
          [ 2.0629, -0.9069, -0.3133,  0.3385, -0.5150, -0.6662],
          [-0.9954, -1.2594,  0.3948,  1.3180, -0.5607,  1.1027]]],
        grad_fn=<NativeLayerNormBackward0>),
 tensor([[[[1.1111e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
           [0.0000e+00, 8.7347e-01, 0.0000e+00, 0.0000e+00],
           [0.0000e+00, 2.2481e-0

# 5. PositionWiseFeedForward 前馈全连接层
什么是前馈全连接层:
在 Transformer 中前馈全连接层就是具有两层线性层的全连接网络。然后使用ReLU函数激活

In [19]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        
        super(PoswiseFeedForwardNet, self).__init__()

        #两层前馈神经网络，视为用大小为1的卷积核进行两次卷积。第一层大小是 [d_model * d_ff]， 第二层大小是 [d_ff * d_model]
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)        
        
        """也可以用nn.Linear写：
        self.w1 = nn.Linear(d_model, d_ff)
        self.w2 = nn.Linear(d_ff, d_model) 
        """
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        residual = x
        
        # 先经过第一个线性层，并使用ReLU数进行激活,
        output = nn.ReLU()(self.conv1(x.transpose(1, 2)))
        
        # 使用dropout进行随机置0，然后通过第二个线性层，最后还原成 output.size = x.size
        output = self.dropout(output)
        output = self.conv2(output).transpose(1, 2)
             
        # 对FFN的输出会再进行一次层后规范化，返回最终结果.
        return self.layer_norm(output + residual)

实例化代码演示：

In [20]:
attn_content = mha(Q, K, V, sm)[0]
attn_content

tensor([[[ 1.4564,  0.6518, -1.0835,  0.7853, -1.0176, -0.7924],
         [-1.4108,  0.7339,  1.6945, -0.7071,  0.0050, -0.3155],
         [ 1.9047, -0.0078, -0.9826,  0.5226, -0.9433, -0.4936],
         [-1.4707, -0.7171,  1.0369, -0.5046,  1.3858,  0.2697]],

        [[-1.0284, -0.2473,  0.6904, -0.5798,  1.8844, -0.7193],
         [-0.8585,  0.1855,  1.1382, -1.5356,  1.2430, -0.1727],
         [-0.1764, -1.3445, -0.0092,  1.0400, -0.9714,  1.4615],
         [ 0.0575, -1.7678, -0.3048,  0.2341,  0.1362,  1.6448]],

        [[-0.1174, -1.1076, -0.4242,  0.4536, -0.7548,  1.9504],
         [ 1.5176,  0.5587, -1.4686,  0.5155, -0.9686, -0.1545],
         [ 2.0719, -0.8751, -0.2793,  0.3133, -0.5519, -0.6789],
         [-0.9954, -1.2594,  0.3948,  1.3180, -0.5607,  1.1027]]],
       grad_fn=<NativeLayerNormBackward0>)

In [21]:
ffn = PoswiseFeedForwardNet(6, 24) # 一般d_ff为d_model的4倍，如在原论文中，d_model=512, d_ff=2048
ffn(attn_content)

tensor([[[ 1.0829,  0.7232, -1.1901,  0.8422, -1.4760,  0.0178],
         [-1.3094,  1.0618,  1.5885, -0.5987, -0.3883, -0.3540],
         [ 1.4561,  0.0826, -1.0759,  0.6144, -1.4761,  0.3989],
         [-1.5068, -0.5064,  1.3161, -0.7094,  1.0557,  0.3508]],

        [[-1.1044,  0.2847,  0.7378, -0.6940,  1.6864, -0.9106],
         [-0.9071,  0.3679,  1.2522, -1.5933,  0.9636, -0.0833],
         [ 0.0603, -0.9050,  0.0323,  1.0002, -1.5358,  1.3479],
         [ 0.3716, -1.4460, -0.3464,  0.3112, -0.6572,  1.7669]],

        [[ 0.1538, -0.6037, -0.4025,  0.5885, -1.4600,  1.7239],
         [ 1.1638,  0.7383, -1.4932,  0.4402, -1.2359,  0.3868],
         [ 1.7419, -1.0222, -0.2820,  0.5362, -1.2217,  0.2478],
         [-0.6675, -0.7780,  0.4567,  1.3883, -1.3688,  0.9693]]],
       grad_fn=<NativeLayerNormBackward0>)

# 6.1 Encoder Layer 编码器层

In [22]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model=512, n_heads=8, d_ff=2048, dropout=0.1):
        super(EncoderLayer, self).__init__()
        
        self.enc_self_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.pos_ffn = PoswiseFeedForwardNet(d_model, d_ff, dropout)

    def forward(self, enc_inputs, enc_self_attn_mask):
        ## 下面这个就是做自注意力层，输入是enc_inputs即pe_results，形状是[batch_size x seq_len_q x d_model] 
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

In [23]:
ecl = EncoderLayer(d_model=6, n_heads=3, d_ff=24, dropout=0.01)
sm = subsequent_mask(4)
sm = sm.unsqueeze(0).repeat(3,3,1,1)
eecl = ecl(pe_result, sm)
eecl[0]

tensor([[[ 1.5300,  0.8344, -1.6425, -0.1301, -0.1051, -0.4868],
         [-0.8883,  1.3666,  0.5628, -1.4410,  0.8595, -0.4597],
         [ 2.0134,  0.1067, -1.0790,  0.1674, -0.5263, -0.6823],
         [-1.0571,  0.2084,  0.3639, -1.1138,  1.8453, -0.2468]],

        [[-0.7764,  0.0455,  0.6984, -0.0477,  1.6057, -1.5254],
         [-0.4818,  0.5994,  0.6658, -1.2368,  1.5176, -1.0642],
         [-0.1171, -1.0033,  1.5803,  0.6792, -1.3978,  0.2588],
         [ 0.4569, -1.9593,  1.0140, -0.2151, -0.2098,  0.9132]],

        [[-0.3552, -0.4371,  1.1087, -0.7873, -1.1289,  1.5998],
         [ 1.7289,  0.9740, -0.9175, -0.2759, -0.7033, -0.8061],
         [ 2.1260, -0.4349,  0.0495, -0.3372, -0.3935, -1.0099],
         [-0.4993, -1.6306,  1.0758,  0.4691, -0.5859,  1.1709]]],
       grad_fn=<NativeLayerNormBackward0>)

# 6.2 Encoder 编码器架构
包含embedding, pos_encoding, 以及N=6层的 encoding_layer

In [24]:
class Encoder(nn.Module):
    def __init__(self, N=6, d_model=512, d_ff=2048, n_heads=8, use_attn_mask=True, dropout=0.1, vocab=200, max_len=500):
        super(Encoder, self).__init__()
        self.n_heads = n_heads
        
        # 初始化输入嵌入层和位置编码层
        self.emb = Embeddings(vocab,d_model)
        self.pe = PositionalEncoding(d_model, dropout, max_len)
        
        # 初始化N个encoder层，使用nn.ModuleList类型的列表进行存储
        self.layers = nn.ModuleList([ EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(N) ])
        
        # 是否制作掩码张量
        self.use_attn_mask = use_attn_mask
        
    def forward(self, x):      
        #将原始输入进行embedding和位置编码
        enc_outputs = self.pe( self.emb(x) )
        
        
        # 制作掩码张量
        """ attn_mask.size = [batch_size * n_heads * len_q * len_k] """
        if self.use_attn_mask:
            batch_size, len_q = x.size()
            attn_mask = subsequent_mask(len_q).unsqueeze(1).repeat(batch_size, self.n_heads, 1, 1)
        else:
            attn_mask = None
        
        # 前向传播
        self_attns = []
        for layer in self.layers:
            enc_outputs, self_attn = layer(enc_outputs, attn_mask)
            self_attns.append(self_attn)
            
        return enc_outputs, self_attns  

编码器实例化演示

In [25]:
encoder = Encoder(d_model=6, d_ff=24, n_heads=3, use_attn_mask=False)
print(encoder(x)[0].size())
len(encoder(x)[1])

torch.Size([3, 4, 6])


6

# 7.1 Decoder Layer 解码器层 

每个解码器层由三个子层连接结构组成：\
第一个子层连接结构包括一个多头自注意力子层（Q=K=V）以及一个残差连接&规范化 \
第二个子层连接结构包括一个多头注意力子层（Q!=K=V）以及一个残差连接&规范化 \
第三个子层连接结构包括一个前馈全连接子层以及一个残差连接&规范化

In [26]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model=512, n_heads=8, d_ff=2048, dropout=0.1):
        super(DecoderLayer, self).__init__()
        
        # 初始化三个子层
        self.dec_self_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.dec_enc_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.pos_ffn = PoswiseFeedForwardNet(d_model, d_ff, dropout)
        
    def forward(self, dec_inputs, enc_outputs, self_attn_mask=None, enc_attn_mask=None):
        
        #掩码多头注意力层
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, self_attn_mask)
        
        # 多头注意力层, 其中Q矩阵是上一子层的dec_outputs, 而encoder的输出作为这一层的K和V
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, enc_attn_mask)
        
        # 前馈神经网络层
        dec_outputs = self.pos_ffn(dec_outputs)
        
        return dec_outputs, dec_self_attn, dec_enc_attn

解码层实例化演示

In [27]:
dec_x = torch.LongTensor([[2,1,0,0],[2,0,2,4],[3,1,3,2]])
declayer = DecoderLayer(d_model=6, n_heads=3, d_ff=24)
declayer( pe(emb(dec_x)), encoder(x)[0]    )[0]

tensor([[[-1.0726, -1.0023, -0.2887,  0.4433,  0.0325,  1.8879],
         [ 1.8313, -0.0489, -1.4613,  0.2405, -0.6637,  0.1020],
         [-0.8560,  0.2194, -0.2083, -0.9397,  2.0542, -0.2697],
         [-0.8977, -0.1526,  0.9212, -0.6570,  1.7313, -0.9452]],

        [[-0.9028, -1.7602,  0.4903,  0.6414,  0.4033,  1.1279],
         [-0.8308,  0.4899,  1.2413, -0.8073,  1.1519, -1.2450],
         [-0.0362, -1.7682,  0.5977,  0.5558, -0.6731,  1.3240],
         [-0.0792, -1.7173, -0.5125,  0.4751,  0.2555,  1.5783]],

        [[ 0.1103, -1.6690, -0.2113, -0.1845,  0.1984,  1.7562],
         [ 1.7723, -0.1663, -0.8515,  0.7531, -1.2024, -0.3052],
         [ 0.3010, -1.8959, -0.1438, -0.1485,  0.4480,  1.4392],
         [ 0.8718, -0.6511, -0.2046,  0.4272, -1.7206,  1.2772]]],
       grad_fn=<NativeLayerNormBackward0>)

# 7.2 Decoder 解码器架构

In [28]:
# 定义一个新的掩码器函数，使得掩码张量可以 len_q!=len_k
def dec_source_mask(len_q, len_k):
    attn_shape = (1, len_q, len_k)

    # 然后使用np.ones方法向这个形状中添加1元素,形成上三角阵, 最后为了节约空间, 
    # 再使其中的数据类型变为无符号8位整形unit8 
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')

    # 最后将三角阵反转
    return torch.from_numpy(1 - subsequent_mask)

dec_source_mask(5,7)

tensor([[[1, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0]]], dtype=torch.uint8)

In [51]:
class Decoder(nn.Module):
    def __init__(self, N=6, d_model=512, d_ff=2048, n_heads=8, use_enc_attnmask=True, dropout=0.1, vocab=200, max_len=500):
        super(Decoder, self).__init__()
        self.n_heads = n_heads
        
        # 初始化输入嵌入层和位置编码层
        self.emb = Embeddings(vocab,d_model)
        self.pe = PositionalEncoding(d_model, dropout, max_len)
        
        # 初始化N个decoder层，使用nn.ModuleList类型的列表进行存储
        self.layers = nn.ModuleList([ DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(N) ])
        
        # 决定每个decoder layer中的第二小层是否使用掩码张量（第一小层必须要用）
        self.use_enc_attnmask = use_enc_attnmask
           
        
    def forward(self, dec_inputs, enc_inputs, enc_outputs): # dec_inputs : [batch_size x target_len]    
        #将原始输入进行embedding和位置编码
        dec_outputs = self.pe( self.emb(dec_inputs) )
        
        
        # 制作掩码张量
        """
        在第一个子层中，必须要用msak，即要对目标数据进行遮掩，因为此时模型可能还没有生成任何目标数据，
        比如在解码器准备生成第一个字符或词汇时，我们其实已经传入了第一个字符以便计算损失，
        但是我们不希望在生成第一个字符时模型能利用这个信息，因此我们会将其遮掩，同样生成第二个字符或词汇时，
        所以模型只能使用第一个字符或词汇信息，第二个字符以及之后的信息都不允许被模型使用.
        self_attn_mask.size = [batch_size * n_heads * len_q * len_k]，其中 len_q = len_k
        """
        batch_size, len_q = dec_inputs.size()
        len_k = enc_inputs.size()[1]
        self_attn_mask = subsequent_mask(len_q).unsqueeze(1).repeat(batch_size, self.n_heads, 1, 1)       
               
            
        # 接着进入第二个子层，这个子层中常规的注意力机制，q是输入x; k，v是编码层输出memory， 
        # 同样也传入source_mask，但是进行源数据遮掩的原因并非是抑制信息泄漏，而是遮蔽掉对结果没有意义的字符而产生的注意力值，
        # 以此提升模型效果和训练速度. 这样就完成了第二个子层的处理. 故第二个子层可以不用掩码           
        """ enc_attn_mask.size = [batch_size * n_heads * len_q * len_k]，其中 len_q != len_k """ 
        if self.use_enc_attnmask:  
            enc_attn_mask = dec_source_mask(len_q, len_k).unsqueeze(1).repeat(batch_size, self.n_heads, 1, 1)
        else:
            enc_attn_mask = None
        
        # 前向传播
        dec_self_attns, dec_enc_attns = [], []
        
        for layer in self.layers:
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, self_attn_mask, enc_attn_mask)
            
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        return dec_outputs, dec_self_attns, dec_enc_attns

代码实例化演示

In [30]:
x = torch.LongTensor([[1,0,4,1],[0,1,2,0],[3,2, 1,0]])
encoder = Encoder(d_model=6, d_ff=24, n_heads=3, use_attn_mask=False)
enc_outputs = encoder(x)[0]

In [31]:
enc_outputs.size()

torch.Size([3, 4, 6])

In [42]:
dec_inputs = torch.LongTensor([[2,1,0,0,2],[3,2,0,2,4],[1,3,1,3,2]]) # now len_q=5, len_k=4
decoder = Decoder(d_model=6, d_ff=24, n_heads=3, use_enc_attnmask=False)
dec_outputs = decoder(dec_inputs, x, enc_outputs)
dec_outputs[0]

tensor([[[-1.0595,  1.6973, -0.6975, -0.6056, -0.3463,  1.0116],
         [-1.4779,  1.5660, -0.0222, -0.7089, -0.2501,  0.8931],
         [ 0.2229,  0.3392,  1.6736, -1.5056, -0.8656,  0.1355],
         [ 0.5927, -0.1060,  1.5278, -1.7943, -0.2827,  0.0624],
         [-1.7495,  0.8717,  0.0192, -0.7810,  0.4844,  1.1552]],

        [[-0.1564, -1.2217, -0.5915,  1.6784,  0.9436, -0.6525],
         [-0.3055,  0.2272,  0.7266, -1.7239,  1.4803, -0.4047],
         [ 1.0429, -0.8495,  1.6297, -0.7453, -0.0924, -0.9854],
         [-0.3373, -0.2143,  0.7341, -1.9226,  1.0832,  0.6570],
         [-1.6081,  0.5620,  1.1555, -0.0414, -0.9718,  0.9039]],

        [[-1.9504,  0.3669,  0.4542, -0.4437,  0.3276,  1.2453],
         [-0.4618, -1.2197, -1.1767,  1.2612,  0.6429,  0.9541],
         [-1.7562, -0.0398,  0.2902, -0.4920,  0.4546,  1.5431],
         [-0.6922, -1.2165, -0.9897,  1.3708,  0.8525,  0.6751],
         [-0.5430,  0.3944,  0.1795, -1.7377,  0.1320,  1.5748]]],
       grad_fn=<Nat

# 7.3 Generator 生成器层（输出层）
包含一个线性层和一个softmax层。线性层的作用是通过线性变化得到指定维度的输出，也就是转换维度的作用。\
最终输出维度是 [batch_size * len_decoderinput * output_vocab_size]

In [43]:
class Generator(nn.Module):
    def __init__(self, d_model, vocab_size):
        super(Generator, self).__init__()
        # 初始化线性层
        self.project = nn.Linear(d_model, vocab_size)

    def forward(self, x):

        return nn.Softmax(dim=-1)(self.project(x)) #F.log_softmax(self.project(x), dim=-1)

In [47]:
# 实例化演示
ge = Generator(6, 15)
ge(dec_outputs[0]).size()

torch.Size([3, 5, 15])

# 8. Transformer架构

In [48]:
class Transformer(nn.Module):
    def __init__(self, enc_N=6, d_model=512, n_heads=8, d_ff=2048, enc_vocab=200, enc_attnmask=True, 
                 dec_N=6, dec_tgt_mask=True, dec_vocab=200, dropout=0.1,  max_len=500):
        super(Transformer, self).__init__()
        
        ## 初始化编码层
        self.encoder = Encoder(enc_N, d_model, d_ff, n_heads, enc_attnmask, dropout, enc_vocab, max_len)
        
        ## 初始化解码层
        self.decoder = Decoder(dec_N, d_model, d_ff, n_heads, dec_tgt_mask, dropout, enc_vocab, max_len) 
        
        ## 初始化输出层 
        self.generator = Generator(d_model, dec_vocab) 
    
    
    def forward(self, enc_inputs, dec_inputs):
        
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)

        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)

        transformer_outputs = self.generator(dec_outputs)
        
        return transformer_outputs, enc_self_attns, dec_self_attns, dec_enc_attns

In [59]:
enc_inputs = torch.LongTensor([[1,0,4,1],[0,1,2,0],[3,2,1,0]])
dec_inputs = torch.LongTensor([[2,1,0,0,2],[3,2,0,2,4],[1,3,1,3,2]])

tran = Transformer(d_model=24, n_heads=6, d_ff=96, enc_attnmask=False, dropout=0.01, dec_vocab=7)
result = tran(enc_inputs, dec_inputs)
result[0]

tensor([[[0.1027, 0.0689, 0.3123, 0.1395, 0.0856, 0.0695, 0.2215],
         [0.1202, 0.1166, 0.2708, 0.1569, 0.0554, 0.0682, 0.2119],
         [0.1124, 0.1425, 0.3220, 0.2132, 0.0394, 0.0555, 0.1151],
         [0.1114, 0.1526, 0.3032, 0.2259, 0.0382, 0.0583, 0.1104],
         [0.0736, 0.1012, 0.1914, 0.1880, 0.0391, 0.1056, 0.3013]],

        [[0.1657, 0.1042, 0.1248, 0.0819, 0.0668, 0.1216, 0.3351],
         [0.0985, 0.1337, 0.0758, 0.0850, 0.0473, 0.1311, 0.4286],
         [0.1564, 0.2175, 0.1698, 0.1935, 0.0433, 0.0746, 0.1449],
         [0.1188, 0.1131, 0.1143, 0.1496, 0.0439, 0.1041, 0.3563],
         [0.1486, 0.0866, 0.1707, 0.0905, 0.0512, 0.1075, 0.3449]],

        [[0.1717, 0.2157, 0.1611, 0.0985, 0.0405, 0.1591, 0.1535],
         [0.2576, 0.1647, 0.1217, 0.0769, 0.0440, 0.1251, 0.2100],
         [0.1303, 0.2547, 0.1184, 0.0896, 0.0439, 0.1330, 0.2300],
         [0.2119, 0.1446, 0.1438, 0.0777, 0.0497, 0.1317, 0.2406],
         [0.0986, 0.1458, 0.0903, 0.0773, 0.0312, 0.1530, 

开始循环迭代训练

In [80]:
import torch.optim as optim

if __name__ == '__main__':
    enc_inputs = torch.LongTensor([[1,0,4,1],[0,1,2,0],[3,2,1,0]])
    dec_inputs = torch.LongTensor([[2,1,0,0,2],[3,2,0,2,4],[1,3,1,3,2]])

    # 假设这是你所期待的输出（训练样本Y）
    target_tensor = torch.LongTensor([[1,5,3,1,0],[2,2,3,1,4],[3,4,2,2,3]])
    
    model = Transformer(d_model=512, n_heads=8, d_ff=2048, dropout=0.01, dec_vocab=7)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam( model.parameters(), lr=0.001)  # 在这里transformer内部神经网络的weight和bias就是parameters
    
    for epoch in range(10):
        optimizer.zero_grad()
        
        outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
        
        # 计算损失函数
        loss = criterion( outputs.view(-1, outputs.size(-1)),  target_tensor.contiguous().view(-1))
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
        
        loss.backward()
        optimizer.step()


Epoch: 0001 cost = 1.933191
Epoch: 0002 cost = 1.794412
Epoch: 0003 cost = 1.930727
Epoch: 0004 cost = 1.898750
Epoch: 0005 cost = 1.898753
Epoch: 0006 cost = 1.898753
Epoch: 0007 cost = 1.898754
Epoch: 0008 cost = 1.898754
Epoch: 0009 cost = 1.898754
Epoch: 0010 cost = 1.898754


 更多的模型训练过程，可以参考 https://sliu.vip/machine-learning/transformer-build/
    
    使用 copy 任务进行模型基本测试的四步曲 
    第一步：构建数据集生成器 
    第二步：获得 Transformer 模型及其优化器和损失函数 
    第三步：运行模型进行训练和评估 
    第四步：使用模型进行贪婪解码 

In [65]:
sentences = ['ich mochte ein bier P', 'S i want a beer', 'i want a beer E']


# Transformer Parameters
# Padding Should be Zero
## 构建词表
src_vocab = {'P': 0, 'ich': 1, 'mochte': 2, 'ein': 3, 'bier': 4}
src_vocab_size = len(src_vocab)

tgt_vocab = {'P': 0, 'i': 1, 'want': 2, 'a': 3, 'beer': 4, 'S': 5, 'E': 6}
tgt_vocab_size = len(tgt_vocab)


def make_batch(sentences):
    input_batch = [[src_vocab[n] for n in sentences[0].split()]]
    output_batch = [[tgt_vocab[n] for n in sentences[1].split()]]
    target_batch = [[tgt_vocab[n] for n in sentences[2].split()]]
    return torch.LongTensor(input_batch), torch.LongTensor(output_batch), torch.LongTensor(target_batch)

make_batch(sentences)

(tensor([[1, 2, 3, 4, 0]]),
 tensor([[5, 1, 2, 3, 4]]),
 tensor([[1, 2, 3, 4, 6]]))

In [ ]:
nn.linear在做什么？和1D cnn卷积的区别？
什么是sublayer(x)?